In [200]:
import scipy.io
import numpy as np
import random
import time
import datetime

In [224]:
a=time.time();b=a=time.time();a-b

0.0

In [3]:
mat = scipy.io.loadmat('/Users/yzh/Desktop/cour/supervised/HW1_Data/dataset1.mat')

In [9]:
mat.keys()

dict_keys(['Y_trn', 'X_tst', 'Y_tst', 'X_trn', '__header__', '__version__', '__globals__'])

In [164]:
def getPolyMat(input, power): #get poly feature matrix
  result=np.power(input,0)
  for i in range(1,power+1):
      result=np.concatenate((result,np.power(input,i)),axis=1)
  return result

In [169]:
mat['X_trn']

array([[ 1.41883768],
       [-3.08617234],
       [-2.33266533],
       [-2.38076152],
       [-0.53707415],
       [ 3.03807615],
       [ 2.66933868],
       [ 0.92184369],
       [ 2.49298597],
       [-3.90380762],
       [-2.09218437],
       [-0.2004008 ],
       [-2.90981964],
       [-1.8997996 ],
       [ 0.6492986 ],
       [-3.7755511 ],
       [-2.17234469],
       [ 3.34268537],
       [ 1.13026052],
       [-1.16232465],
       [ 0.90581162],
       [ 0.77755511],
       [ 0.29659319],
       [ 1.56312625],
       [ 1.498998  ],
       [-3.3747495 ],
       [ 1.06613226],
       [-3.74348697],
       [-2.34869739],
       [-2.42885772],
       [ 3.18236473],
       [-1.64328657],
       [ 2.55711423],
       [-1.35470942],
       [ 3.93587174],
       [-0.13627255],
       [-1.14629259],
       [ 1.22645291],
       [ 3.51903808],
       [-0.61723447],
       [-2.79759519],
       [-1.56312625],
       [ 1.32264529],
       [-1.94789579],
       [-3.35871743],
       [ 2

In [167]:
train_X=getPolyMat(mat['X_trn'], 2)

In [95]:
def closeFormRegression(train, label):
    return np.matmul(np.matmul( np.linalg.inv( np.matmul(train.transpose(), train) ),  train.transpose() ), label)

In [146]:
theta=closeFormRegression(train_X, mat['Y_trn'])

In [150]:
theta

array([[ 9.49203678],
       [ 4.79191663],
       [ 1.52906587]])

In [156]:
def GradDesc(train, label, learnRate, stopWhen, batchSize):# stochastic GD
    theta1=np.zeros((train.shape[1],1))
    while(True):
      batch=random.sample(range(0,label.shape[0]),batchSize)
      theta2 = theta1 - learnRate * np.matmul(train.transpose(), np.matmul(train, theta1)- label)
      if np.linalg.norm(theta2-theta1)<=stopWhen:
          return theta2
      theta1=theta2

In [157]:
GradDescTheta=GradDesc(train_X, mat['Y_trn'], 0.00001, 0.0001, 30)

In [158]:
GradDescTheta

array([[ 9.30054836],
       [ 4.79368801],
       [ 1.54895926]])

In [162]:
print(np.linalg.norm( np.matmul(train_X,GradDescTheta)-mat['Y_trn'] )/mat['Y_trn'].shape[0]**0.5)#train error

4.97573589964


In [171]:
test_X=getPolyMat(mat['X_tst'],2)

In [194]:
print(np.linalg.norm( np.matmul(test_X,GradDescTheta)-mat['Y_tst'] )/mat['Y_tst'].shape[0]**0.5)#test error

66.7272651613


In [195]:
np.matmul(test_X,GradDescTheta)-mat['Y_tst']

array([[  4.29493991e+00],
       [  9.21188757e+00],
       [  2.59443250e+01],
       [  1.72976566e+02],
       [ -8.83195394e+01],
       [ -4.16239499e+00],
       [  2.41985964e+01],
       [ -2.88429172e+01],
       [  1.73069874e+01],
       [  4.22130540e+00],
       [ -6.27556693e+00],
       [ -5.80134704e+00],
       [ -1.53802158e+01],
       [  1.35308453e+02],
       [  6.72211462e+01],
       [ -3.08746675e+00],
       [ -5.64855291e+00],
       [ -5.24551217e+01],
       [  1.14515305e+02],
       [  1.94482436e+02],
       [  1.83479282e+02],
       [  1.60472424e+02],
       [ -2.38893878e+01],
       [ -4.24280172e+01],
       [ -5.38114478e+00],
       [  3.66389470e+01],
       [ -4.45614036e+01],
       [ -2.00514055e+01],
       [ -4.04315192e+00],
       [  2.43473444e+00],
       [ -2.14390184e+01],
       [  2.39922363e+01],
       [  2.87063805e+01],
       [ -3.60257479e+00],
       [  1.07724086e+02],
       [  2.69313095e+01],
       [  7.70335542e+00],
 

In [192]:
mat['Y_trn'].shape

(120, 1)

In [ ]:
print

In [93]:
a = np.array([[1, 2], [3, 4]]);
a.transpose()
print(a)

[[1 2]
 [3 4]]
